In [1]:
from gpt import GPTConfig, GPT, generate
import torch
import tokenizers
import os

In [2]:
candidates = []
for i in os.listdir('logs'):
    if 'smoke' in i:
        continue
    best_loss_for_i = 10000
    best_line = ''
    for line in open('logs/'+i+'/log.txt').readlines():

        if 'val' in line and 'byte loss' in line:
            after_bl_str = line.split('byte loss')[1]
            
            try:                
                byte_loss = float(after_bl_str.split(' ')[1])
            except ValueError as e:
                byte_loss = 100000
            if byte_loss < best_loss_for_i:
                best_loss_for_i = byte_loss
                best_line = line
    
    if best_loss_for_i < 10000:
        candidates.append((best_loss_for_i, i, best_line))
candidates = sorted(candidates, key=lambda x: x[0])
for e, f, l in candidates:
    print(f.ljust(50), f'{e:.3} {l.strip()}')

12l_16h_512d_8k_vocab_2e3_lr_good_max_more_warmup  0.374 step 9681 | val loss 1.5355 | byte loss 0.3736 | ds 600.5s
10l_16h_512d_8k_vocab                              0.377 step 11500 | val loss 1.5478 | byte loss 0.3766 | ds 594.2s
12l_16h_512d_8k_vocab                              0.378 step 9881 | val loss 1.5521 | byte loss 0.3776 | ds 600.7s
16l_12h_384d_8k_vocab_2e3_lr_good_max_beta_hax     0.381 step 9349 | val loss 1.5660 | byte loss 0.3810 | ds 589.1s
10l_16h_512d_4k_vocab                              0.382 step 11500 | val loss 1.5065 | byte loss 0.3816 ds 593.7s
12l_32h_512d_8k_vocab_2e3_lr_good_max_eos          0.382 step 7891 | val loss 1.5621 | byte loss 0.3819 | ds 601.0s
10l_12h_384d_more_lr_less_decay                    0.384 step 13999 | val loss 1.5789 | byte loss 0.3841 ds 555.5s
16l_12h_384d_8k_vocab_2e3_lr_bad_max               0.388 step 9311 | val loss 1.5942 | byte loss 0.3879 | ds 600.9s
12l_12h_384d_less_feedfoward_more_lr               0.391 step 12250 | val

In [4]:
all_stuff = torch.load('logs/12l_32h_512d_8k_vocab_2e3_lr_good_max_eos/model_07891.pt')

def remove_orig_mod_prefix(state_dict):
    return {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}

m = GPT(GPTConfig())
m.load_state_dict(remove_orig_mod_prefix(all_stuff['model']))
m.to('cuda')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 512)
    (wpe): Embedding(256, 512)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=1024, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=1024, out_features=512, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=8192, bias=False)
)

In [5]:
data_dir = 'tiny-stories-8k-eos'
enc = tokenizers.ByteLevelBPETokenizer(f'{data_dir}/tokenizer-vocab.json', f'{data_dir}/tokenizer-merges.txt')



In [6]:
generate(m, enc, "Lily went to the park and saw a friendly dog.", 255, 4)

sample 0: Lily went to the park and saw a friendly dog. She wanted to pet it. It was fluffy and brown. She saw the dog. Lily did not know that the dog was not a threatening to get caught. She asked her mom and dad about the catcher. They said hello at the same time.

Lily was so happy. After a little while playing with her new friend. They hugged and the birds. They were very small but it was the perfect!
Lily was so happy. When they saw the park every day. She went back home. One day, Max ran to the park. He saw his mommy and they were gone.

Max didn't like this started to feel really sad. He felt much better. He didn't want to go home that day. He knew that his mom would never forget him.
sample 1: Lily went to the park and saw a friendly dog. The dog was very big and had a big red crayon. She liked the dog, so she asked the dog if she could pet him too. But only for a little while. Lily was three years old. She had so much fun playing games around in the park. She ran and laughed, 